In [40]:
import requests
from requests import get
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
import os.path
from os import path
from pathlib import Path



In [41]:
#Compiling URLs to scrape
years = list(range(2000, 2022))
years_custom = list(range(2000, 2022))
url_big12 = 'https://www.sports-reference.com/cbb/conferences/big-12/{}-stats.html'
url_sec = 'https://www.sports-reference.com/cbb/conferences/sec/{}-stats.html'
url_big10 = 'https://www.sports-reference.com/cbb/conferences/big-ten/{}-stats.html'
url_bigeast = 'https://www.sports-reference.com/cbb/conferences/big-east/{}-stats.html'
url_pac12 = 'https://www.sports-reference.com/cbb/conferences/pac-12/{}-stats.html'
url_acc = 'https://www.sports-reference.com/cbb/conferences/acc/{}-stats.html'
url_aac = 'https://www.sports-reference.com/cbb/conferences/aac/{}-stats.html'
url_mwc = 'https://www.sports-reference.com/cbb/conferences/mwc/{}-stats.html'
url_wcc = 'https://www.sports-reference.com/cbb/conferences/wcc/{}-stats.html'
url_a10 = 'https://www.sports-reference.com/cbb/conferences/atlantic-10/{}-stats.html'
url_mvc = 'https://www.sports-reference.com/cbb/conferences/mvc/{}-stats.html'
url_cusa = 'https://www.sports-reference.com/cbb/conferences/cusa/{}-stats.html'
url_col = 'https://www.sports-reference.com/cbb/conferences/colonial/{}-stats.html'
url_south = 'https://www.sports-reference.com/cbb/conferences/southern/{}-stats.html'
url_wac = 'https://www.sports-reference.com/cbb/conferences/wac/{}-stats.html'
url_maac = 'https://www.sports-reference.com/cbb/conferences/maac/{}-stats.html'
url_bw = 'https://www.sports-reference.com/cbb/conferences/big-west/{}-stats.html'
url_sb = 'https://www.sports-reference.com/cbb/conferences/sun-belt/{}-stats.html'
url_ivy = 'https://www.sports-reference.com/cbb/conferences/ivy/{}-stats.html'
url_mac = 'https://www.sports-reference.com/cbb/conferences/mac/{}-stats.html'
url_as = 'https://www.sports-reference.com/cbb/conferences/atlantic-sun/{}-stats.html'
url_ovc = 'https://www.sports-reference.com/cbb/conferences/ovc/{}-stats.html'
url_summit = 'https://www.sports-reference.com/cbb/conferences/summit/{}-stats.html'
url_big_south = 'https://www.sports-reference.com/cbb/conferences/big-south/{}-stats.html'
url_big_sky = 'https://www.sports-reference.com/cbb/conferences/big-sky/{}-stats.html'
url_america_east = 'https://www.sports-reference.com/cbb/conferences/america-east/{}-stats.html'
url_horizon = 'https://www.sports-reference.com/cbb/conferences/horizon/{}-stats.html'
url_patriot = 'https://www.sports-reference.com/cbb/conferences/patriot/{}-stats.html'
url_southland = 'https://www.sports-reference.com/cbb/conferences/southland/{}-stats.html'
url_northeast = 'https://www.sports-reference.com/cbb/conferences/northeast/{}-stats.html'
url_meac = 'https://www.sports-reference.com/cbb/conferences/meac/{}-stats.html'
url_swac = 'https://www.sports-reference.com/cbb/conferences/swac/{}-stats.html'
url_draft =     'https://www.nbadraft.net/{}-nba-draft-combine-measurements/'
url_draft_summary = 'https://basketball.realgm.com/nba/draft/past_drafts/{}'

list_of_urls = [
url_big12, 
url_sec, 
url_big10,
url_bigeast,
url_pac12,
url_acc,
url_aac,
url_mwc,
url_wcc,
url_a10,
url_mvc,
url_cusa,
url_col,
url_south,
url_wac,
url_maac,
url_bw,
url_sb,
url_ivy,
url_mac,
url_as,
url_ovc ,
url_summit,
url_big_south,
url_big_sky,
url_america_east,
url_horizon,
url_patriot,
url_southland,
url_northeast,
url_meac,
url_swac
]
#Table ID
NCAA_id = 'conference-stats'

In [42]:
#Generates File name
def generate_name(url, year):
    end_index = url[49:].find('/')
    name = url[49:49+end_index]+'_'+str(year)
    return(name)
    
        
    

In [43]:
#Generates File Directory
def generate_directory(directory, file_name):
    generated_directory = directory + '/{}.html'.format(file_name)
    return(generated_directory)
    

In [44]:
#Downloads HTML File from given URL to directory
def get_html_request(directory, url):
    for year in years:
        url_temp = url.format(year)
        data = requests.get(url_temp)
        with open(generate_directory(directory, generate_name(url, year)), 'w+',encoding="utf-8") as f:
                f.write(data.text)

In [45]:
#Downloads HTML File from given URL to /NCAA
def get_html_request_NCAA(url):
    for year in years:
        url_temp = url.format(year)
        data = requests.get(url_temp)
        with open('NCAA/{}.html'.format(generate_name(url, year)), 'w+',encoding="utf-8") as f:
                f.write(data.text)

In [46]:
#Downloads HTML File from given URL to /Draft
def get_html_request_draft(url):
    for year in years:
        url_temp = url.format(year)
        data = requests.get(url_temp)
        with open('Draft/{}.html'.format(year), 'w+',encoding="utf-8") as f:
                f.write(data.text)

In [47]:
#Downloads HTML File from given URL to /NCAA, uses Selenium to execute javascript if required
def get_html_api(url):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    for year in years_custom:        
        temp_url = url.format(year)
        driver.get(temp_url)
        driver.execute_script("window.scrollTo(1,10000)")
        time.sleep(4)
        with open("NCAA/{}.html".format(generate_name(url, year)), "w+",encoding="utf-8") as f:
            f.write(driver.page_source)
    driver.quit()

In [48]:
#Searches webpage for all tables
def search_url_for_tables(url):
    r = get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    tables = soup.find_all('table')
    return(tables)

In [49]:
#Uses requests to get html and searches for all tables, returning them as a df with first row as header
def get_df_from_url(url):
    r = get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    tables = soup.find_all('table')
    df = pd.read_html(str(tables))[0]
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data unless the header row
    df.columns = new_header
    return(df)

In [50]:
#Uses requests to get html and searches for all tables, returning them as a df with no header
def get_df_from_url_noheader(url):
    r = get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    tables = soup.find_all('table')
    df = pd.read_html(str(tables))[0]
    return(df)

In [51]:
#Opens html file and extracts table to df from table id
def html_to_df_from_tableid(file_name, table_id):
    with open('NCAA/{}.html'.format(file_name), encoding="utf-8") as f:
        page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        soup.find('tr', class_= 'over_header').decompose() #Removes 'over_header' element of table
        table = soup.find(id=table_id)
        df = pd.read_html(str(table))[0]
    return df

#Uses URL to get df based on table id
def html_to_df_from_tableid_no_download(url, table_id):
    r = get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    table = soup.find(id=table_id)
    df = pd.read_html(str(table))[0]
    return df

#Uses URL to get df based on table class
def html_to_df_from_table_class_no_download(url):
    r = get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    table = soup.find('table', {'class' : ["tablesaw", "tablesaw-swipe", "tablesaw-sortable"]})
    df = pd.read_html(str(table))[0]
    return df



In [52]:
#Removes Positons from player names as to only include player name in 'Player' Collumn
def clean_player_name(series):
    new_series = []
    for item in series:
        index = 0
        for char in reversed(item):
            if char.capitalize() == char or char == '-':
                index = index - 1
            else:
                break
        new_series.append(item[:index].strip())
    return(new_series)

In [53]:
#Converts from inches to cm so model can interpret data
def inch_to_cm(series):
    new_series = []
    for measurement in series:
        measurement = measurement.strip()
        feet = float(measurement[0])
        inches = float(measurement[2:-1])
        height_inches = feet*12 + inches
        height_cm = round(height_inches * 2.54, 2)
        new_series.append(height_cm)
    return(new_series)


In [54]:
#Removes instances in dataset where last character is a non-int value 
def remove_last_char(series):
    new_series = []
    for item in series:
        if isinstance(item, str):
            if item[-1] == '‘' or item[-1] == '′' or item[-1] == '%':
                new_series.append(item[:-1])
            else:
                new_series.append(item)
        else:
            new_series.append(item)
    return(new_series)    

In [72]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import draftcombinestats
import time
from time import sleep

combine_dfs = []
for year in years:
    combine = draftcombinestats.DraftCombineStats(season_all_time = year)
    combine_df = combine.get_data_frames()
    combine_dfs.append(combine_df[0])
    sleep(0.5)
combine_data_df = pd.concat(combine_dfs)

         

In [83]:
list_of_columns = list(combine_data_df.columns)
columns_to_drop =   ['HEIGHT_W_SHOES',
                     'HEIGHT_W_SHOES_FT_IN',
                     'HEIGHT_WO_SHOES_FT_IN',
                     'WINGSPAN_FT_IN',
                     'BODY_FAT_PCT',
                     'HAND_LENGTH',
                     'HAND_WIDTH',
                     'MODIFIED_LANE_AGILITY_TIME',
                     'SPOT_FIFTEEN_CORNER_LEFT',
                     'SPOT_FIFTEEN_BREAK_LEFT',
                     'SPOT_FIFTEEN_TOP_KEY',
                     'SPOT_FIFTEEN_BREAK_RIGHT',
                     'SPOT_FIFTEEN_CORNER_RIGHT',
                     'SPOT_COLLEGE_CORNER_LEFT',
                     'SPOT_COLLEGE_BREAK_LEFT',
                     'SPOT_COLLEGE_TOP_KEY',
                     'SPOT_COLLEGE_BREAK_RIGHT',
                     'SPOT_COLLEGE_CORNER_RIGHT',
                     'SPOT_NBA_CORNER_LEFT',
                     'SPOT_NBA_BREAK_LEFT',
                     'SPOT_NBA_TOP_KEY',
                     'SPOT_NBA_BREAK_RIGHT',
                     'SPOT_NBA_CORNER_RIGHT',
                     'OFF_DRIB_FIFTEEN_BREAK_LEFT',
                     'OFF_DRIB_FIFTEEN_TOP_KEY',
                     'OFF_DRIB_FIFTEEN_BREAK_RIGHT',
                     'OFF_DRIB_COLLEGE_BREAK_LEFT',
                     'OFF_DRIB_COLLEGE_TOP_KEY',
                     'OFF_DRIB_COLLEGE_BREAK_RIGHT',
                     'ON_MOVE_FIFTEEN',
                     'ON_MOVE_COLLEGE']

combine_df = combine_data_df.drop(columns = columns_to_drop)
combine_df = combine_df.dropna()


columns_to_rename = {'PLAYER_NAME': 'Player'}
combine_df = combine_df.rename(columns = columns_to_rename)

In [84]:
combine_df

,SEASON,PLAYER_ID,FIRST_NAME,LAST_NAME,Player,POSITION,HEIGHT_WO_SHOES,WEIGHT,WINGSPAN,STANDING_REACH,STANDING_REACH_FT_IN,STANDING_VERTICAL_LEAP,MAX_VERTICAL_LEAP,LANE_AGILITY_TIME,THREE_QUARTER_SPRINT,BENCH_PRESS
0,2000,2124,Malik,Allen,Malik Allen,PF-C,80.25,271,86.50,109.0,9' 1'',25.5,29.0,11.83,3.38,13.0
2,2000,12020,Lamont,Barnes,Lamont Barnes,PF-C,80.50,235.5,87.50,108.0,9' 0'',28.0,29.5,12.30,3.40,10.0
3,2000,12131,Mario,Bland,Mario Bland,PF,77.50,287,84.00,103.0,8' 7'',27.0,31.0,13.04,3.47,15.0
6,2000,2049,Speedy,Claxton,Speedy Claxton,PG,70.50,166,72.00,94.5,7' 10.5'',36.0,42.5,10.48,3.06,6.0
7,2000,12132,Eric,Coley,Eric Coley,SG-SF,76.00,205,83.00,102.0,8' 6'',35.0,38.0,11.40,3.15,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2019,1629679,Simisola,Shittu,Simisola Shittu,PF,80.50,227.2,85.25,106.0,8' 10'',27.0,35.5,11.85,3.30,11.0
70,2019,1629682,Tremont,Waters,Tremont Waters,PG,69.50,172.4,74.25,93.5,7' 9.5'',30.5,40.5,10.76,3.07,5.0
73,2019,1629025,Kris,Wilkes,Kris Wilkes,SF,78.25,208.8,82.75,103.0,8' 7'',27.5,35.0,10.39,3.34,4.0
74,2019,1629684,Grant,Williams,Grant Williams,PF,77.75,240.2,81.75,104.5,8' 8.5'',26.0,31.5,10.83,3.33,20.0


In [55]:
#Creates a list of dfs of draft data for each year in years_custom
dfs_draft = []
for year in years_custom:
    url = url_draft_summary.format(year)
    dfs = pd.read_html(url)
    temp_list = [dfs[0], dfs[1]]  #Note that https://basketball.realgm.com/ will occasionally change webpage layout and therefore indexing may need to be adjusted
    temp_df = pd.concat(temp_list)
    temp_df['Year'] = year
    dfs_draft.append(temp_df)



#Gets relevant columns from the draft data dataframes
dfs_clean = []
relevant_columns = ['Player', 'Pick', 'Age', 'Year']
for df in dfs_draft:
    df = df[relevant_columns]
    dfs_clean.append(df)

draft_data_df = pd.concat(dfs_clean)
#final = pd.merge(draft_main_df, draft_data_df, on=['Player', 'Year'])
draft_data_df

,Player,Pick,Age,Year
0,Kenyon Martin,1,22,2000
1,Stromile Swift,2,20,2000
2,Darius Miles,3,18,2000
3,Marcus Fizer,4,21,2000
4,Mike Miller,5,20,2000
...,...,...,...,...
25,Scottie Lewis,56,21,2021
26,Balsa Koprivica,57,21,2021
27,Jericho Sims,58,22,2021
28,Raiquan Gray,59,22,2021


In [17]:
#Scrapes around 200 webpages from basketball reference, Saves files in /NCAA
#Can run it if you would like but Selenium may crash every once in a while
#If this occurs I would suggest indexing list_of_urls to avoid scraping pages already scraped before the crash and re-running cell
'''
for url in list_of_urls[]:
    get_html_api(url)
'''
#All files nessecary already in directory

'\nfor url in list_of_urls[]:\n    get_html_api(url)\n'

In [32]:
#Goes through each HTML file in NCAA and extracts table as a df and adds it to the dfs_college list 
#If no table/player data in HTML, file is deleted
#Cell will take around 5 minutes to run
dfs_college = []
for url in list_of_urls:
    for year in years_custom:
        file_name = generate_name(url, year)
        path = Path('./NCAA/{}.html'.format(file_name))
        if os.path.isfile(path):                                       #Checks if file exists
            with open('NCAA/{}.html'.format(file_name), encoding="utf-8") as f:
                page = f.read()
                soup = BeautifulSoup(page, 'html.parser')
                try:                                                   #Creates df from player data table and adds it to a list
                    soup.find('tr', class_= 'over_header').decompose() #This will raise AttributeError if there is no player data on webpage
                    table = soup.find(id=NCAA_id)                      
                    df = pd.read_html(str(table))[0]
                    df['Year'] = year
                    dfs_college.append(df)
                except AttributeError:                                 #If no player data table exists the HTML file is deleted
                    f.close()                                          #This occurs because some conferences did not exist until later 
                    os.remove(path)
            

In [ ]:
#Adds all college players to a single df and gets relevant collumns
college_total = pd.concat(dfs_college)
relevant_columns = ['Player', 'Pos', 'G','MP','FG%','2P%','3P%','FT%','PTS.1','TRB.1','AST.1','STL.1','BLK.1','PER','WS','BPM', 'Year']
columns_to_rename = {'PTS.1': 'PTS',
                    'AST.1': 'AST',
                    'STL.1': 'STL',
                    'BLK.1': 'BLK',
                    'TRB.1': 'TRB'}

college_total_clean = college_total.loc[:,relevant_columns]
college_total_clean.rename(columns = columns_to_rename , inplace = True)

#Merges df containing college player stats and NBA draft info to a single df which is written to total.csv for analysis
total = pd.merge(college_total_clean, draft_data_df, on=['Player', 'Year'])
total['3P%'] = total['3P%'].fillna(0)
total = total.dropna()

total.to_csv('total.csv')

In [68]:
college_total.loc[college_total['Player'] == 'Jordan Bell']

,Rk,Player,Class,Pos,School,G,MP,TRB,AST,STL,...,FT%,PTS.1,TRB.1,AST.1,STL.1,BLK.1,PER,WS,BPM,Year
15,16,Jordan Bell,FR,F,Oregon,35,831,214,47,28,...,.524,5.1,6.1,1.3,0.8,2.7,19.2,2.9,7.1,2015
13,14,Jordan Bell,SO,F,Oregon,31,636,163,38,34,...,.519,6.8,5.3,1.2,1.1,1.7,20.1,2.3,7.0,2016
15,16,Jordan Bell,JR,F,Oregon,39,1124,342,70,49,...,.701,10.9,8.8,1.8,1.3,2.3,26.4,6.2,10.8,2017
10,11,Jordan Bell,SO,F,Loyola Marymount,32,282,57,13,12,...,.824,2.7,1.8,0.4,0.4,0.3,13.2,0.7,-2.6,2019
11,12,Jordan Bell,JR,F,Loyola Marymount,31,687,138,46,30,...,.667,7.2,4.5,1.5,1.0,0.9,16.2,1.6,1.5,2020
16,17,Jordan Bell,SR,F,Pacific,18,461,124,16,21,...,.571,8.9,6.9,0.9,1.2,0.9,18.8,1.7,4.6,2021
9,10,Jordan Bell,FR,F,Northwestern State,26,302,84,16,8,...,.650,4.1,3.2,0.6,0.3,0.4,14.8,0.7,-3.2,2017


In [60]:
#ids 
'''
158 145 Tristan Thompson
95  88  David Lee
51  48  Jalen Harris
118 109 Jalen Johnson
75 70   Carl Landry
213 196 Dion Waiters
15 16   Jordan Bell

'''

'\n158\t145\t\n'

In [88]:
total.to_csv('total.csv')

In [86]:
total

,SEASON,PLAYER_ID,FIRST_NAME,LAST_NAME,Player,POSITION,HEIGHT_WO_SHOES,WEIGHT,WINGSPAN,STANDING_REACH,...,TRB,AST,STL,BLK,PER,WS,BPM,Year,Pick,Age
0,2000,2049,Speedy,Claxton,Speedy Claxton,PG,70.50,166,72.00,94.5,...,5.4,6.0,3.3,0.2,NaN,7.0,NaN,2000,20,22
1,2000,2064,Khalid,El-Amin,Khalid El-Amin,PG,69.00,200,73.50,93.0,...,3.1,5.2,1.7,0.1,NaN,4.4,NaN,2000,34,21
2,2000,2062,A.J.,Guyton,A.J. Guyton,PG-SG,72.75,175,75.75,97.5,...,3.1,2.3,1.0,0.4,NaN,4.3,NaN,2000,32,22
3,2000,2078,Jason,Hart,Jason Hart,PG,73.75,181,78.00,97.5,...,3.0,6.5,1.8,0.1,NaN,3.6,NaN,2000,49,22
4,2000,2077,Mark,Karcher,Mark Karcher,SG,76.00,218,80.00,102.5,...,4.6,2.2,1.8,0.2,NaN,3.7,NaN,2000,48,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,2019,1629678,Admiral,Schofield,Admiral Schofield,SF,76.00,240.6,81.75,102.5,...,6.1,2.0,0.9,0.5,20.1,4.6,6.3,2019,42,22
477,2019,1629621,Marial,Shayok,Marial Shayok,SF-PF,76.50,196.8,84.25,103.5,...,4.9,2.0,0.9,0.2,22.5,4.7,6.4,2019,54,23
478,2019,1629682,Tremont,Waters,Tremont Waters,PG,69.50,172.4,74.25,93.5,...,2.8,5.8,2.9,0.1,21.8,4.2,8.9,2019,51,21
479,2019,1629684,Grant,Williams,Grant Williams,PF,77.75,240.2,81.75,104.5,...,7.5,3.2,1.1,1.5,30.6,7.9,12.7,2019,22,20


In [89]:
total.at('Jason Hart')

TypeError: '_AtIndexer' object is not callable

In [ ]:
total = pd.merge(combine_df, total, on='Player')
total.to_csv('total.csv')